# 자연어 처리

## 언어를 숫자로 인코딩하기

In [3]:
# 토큰화 시작하기
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

sentences = [
    'Today is a sunny day',
    'Today is a rainy day',
    'Is it sunny today?'
]

tokenizer = Tokenizer(num_words = 100) # num_words는 토큰화할 수 있는 단어 개수
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

# today?를 today로 필터링하는 것을 보아 클래스의 유연성 확인 가능
# Tokenizer 클래스의 filter 매개변수로 조정 가능. 기본값은 작은 따옴표 제외 모든 구두점 제거

{'today': 1, 'is': 2, 'a': 3, 'sunny': 4, 'day': 5, 'rainy': 6, 'it': 7}


In [4]:
# 문장을 시퀀스로 바꾸기

sentences = [
    'Today is a sunny day',
    'Today is a rainy day',
    'Is it sunny today?'
]

tokenizer = Tokenizer(num_words = 100) 
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(sentences)

print(sequences)

[[1, 2, 3, 4, 5], [1, 2, 3, 6, 5], [2, 7, 4, 1]]


In [5]:
# 훈련 데이터에 존재하지 않는 단어가 예측 데이터 포함될 경우 예측에 부정적 영향 발생
# 이를 해결하기 위한 방법 중 하나로 OOV(out-of-vocabulary) 토큰 사용

test_data = [
    'Today is a snowy day',
    'Will it be rainy tomorrow?'
]

test_sequences = tokenizer.texts_to_sequences(test_data)
print(word_index)
print(test_sequences)

{'today': 1, 'is': 2, 'a': 3, 'sunny': 4, 'day': 5, 'rainy': 6, 'it': 7}
[[1, 2, 3, 5], [7, 6]]


In [6]:
tokenizer = Tokenizer(num_words = 100, oov_token = '<OOV>')
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

test_sequences = tokenizer.texts_to_sequences(test_data)
print(word_index)
print(test_sequences)

{'<OOV>': 1, 'today': 2, 'is': 3, 'a': 4, 'sunny': 5, 'day': 6, 'rainy': 7, 'it': 8}
[[2, 3, 4, 1, 6], [1, 8, 1, 7, 1]]


In [11]:
# TestVectorization 층 사용
# 최신 텐서플로에서는 텍스트 처리를 위한 새로운 TextVectorization 클래스를 제공
# [UNK] 가 oov 토큰을 의미
tv = keras.layers.TextVectorization(max_tokens = 100)

tv.adapt(sentences)
tv.get_vocabulary()

# 단어 인덱스 0에 해당하는 토큰은 비워져 있는데 일반적으로 단어 인덱스 0은 패딩을 위해서 사용

['', '[UNK]', 'today', 'is', 'sunny', 'day', 'a', 'rainy', 'it']

In [12]:
test_seq = tv(test_data)
test_seq.numpy()

array([[2, 3, 6, 1, 5],
       [1, 8, 1, 7, 1]])

In [13]:
# 패딩 이해하기
# 신경망 훈련시 일반적으로 모든 데이터는 크기가 동일해야 함.
# 하지만 단어를 토큰화하고 문장을 토큰의 시퀀스로 바꾸면 길이가 다른 문제 발생
# 이때 동일한 길이로 맞추기 위해 패딩 사용

sentences = [
    'Today is a sunny day',
    'Today is a rainy day',
    'Is it sunny today?',
    'I really enjoyed walking in the snow today'
]

tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
print(sequences)

[[2, 3, 4, 5, 6], [2, 3, 4, 7, 6], [3, 8, 5, 2], [9, 10, 11, 12, 13, 14, 15, 2]]


In [14]:
# 동일한 길이로 만들기 위해 pad_sequences 함수 사용

from tensorflow.keras.preprocessing.sequence import pad_sequences

padded = pad_sequences(sequences)

print(padded)

# 아래 보면 0을 이용해 패딩이 수행되는데 이것이 토큰 리스트가 1부터 시작하는 이유

[[ 0  0  0  2  3  4  5  6]
 [ 0  0  0  2  3  4  7  6]
 [ 0  0  0  0  3  8  5  2]
 [ 9 10 11 12 13 14 15  2]]


In [15]:
# 짧은 문장을 긴 문장의 길이에 맞춰 시작 부분을 0으로 패딩했는데 이것을 프리패딩(prepadding)이라고 하며 함수의 기본값
# padding 매개변수에서 변경가능

padded = pad_sequences(sequences, padding = 'post')
print(padded)

[[ 2  3  4  5  6  0  0  0]
 [ 2  3  4  7  6  0  0  0]
 [ 3  8  5  2  0  0  0  0]
 [ 9 10 11 12 13 14 15  2]]


In [17]:
# 아주 긴 문장 하나에 맞춰 패딩이 너무 많이 추가되는 것을 방지하는 차원에서 maxlen 매개변수에 최대 길이 설정 가능
# 문장의 시작부분 삭제
padded = pad_sequences(sequences, padding = 'post', maxlen = 6)

print(padded)

[[ 2  3  4  5  6  0]
 [ 2  3  4  7  6  0]
 [ 3  8  5  2  0  0]
 [11 12 13 14 15  2]]


In [18]:
# truncating 매개변수를 활용해 문장 끝 부분 삭제 가능
padded = pad_sequences(sequences, padding = 'post', maxlen = 6, truncating = 'post')

print(padded)

[[ 2  3  4  5  6  0]
 [ 2  3  4  7  6  0]
 [ 3  8  5  2  0  0]
 [ 9 10 11 12 13 14]]


#### 텐서플로는 래그드 텐서(ragged tensor, 길이가 다른 텐서)를 사용해 훈련이 가능하다.

#### 책의 내용보다 고급 방법으로 https://oreil.ly/I1IJW 에서 살펴볼 수 있다.

## 불용어 제거와 텍스트 정제

In [ ]:
# 프로그래밍적으로 텍스트를 정제하는 방식에는 세 가지 주요 단계가 존재
# 먼저 HTML 태그를 제거

from bs4 import BeautifulSoup
soup = BeautifulSoup(sentence)
sentence = soup.get_text()

In [ ]:
# 불용어 제거
# 일반적인 방법은 불용어 리스트를 준비하고 문장에서 이를 제거하며 전처리 하는 방식
stopwords = ['a', 'about', 'above', ... 'yours', 'yourself', 'yourselves']

words = sentence.split()
filtered_sentence = ''

for word in words:
  if word not in stopwords:
    filtered_sentence = filtered_sentence + word + " "
sentences.append(filtered_sentence)

In [ ]:
# 또 다른 작업은 구두점을 제거
# 파이썬 string 패키지의 translate 함수를 사용하면 해결 가능

import string
table = str.maketrans('', '', string.punctuation)
words = sentence.split()
filtered_sentence = ''
for word in words:
  word = word.translate(table)
  if word not in stopwords:
    filtered_sentence = filtered_sentence + word + ""

sentences.append(filtered_sentence)

# 하지만 이 과정에서는 불용어 목록에 대한 문제점 발생
# 축약형인 you'll 은 youll로 바뀌므로 불용어 목록에 대한 갱신이 필요

## 실제 데이터 다루기

In [23]:
# 텐서플로 데이터셋에서 텍스트 가져오기
import tensorflow_datasets as tfds

   = []
train_data = tfds.as_numpy(tfds.load('imdb_reviews', split = 'train'))

for item in train_data:
  imdb_sentences.append(str(item['text']))

In [26]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(imdb_sentences)
sequences = tokenizer.texts_to_sequences(imdb_sentences)

In [27]:
print(tokenizer.word_index)

{'the': 1, 'and': 2, 'a': 3, 'of': 4, 'to': 5, 'is': 6, 'br': 7, 'in': 8, 'it': 9, 'i': 10, 'this': 11, 'that': 12, 'was': 13, 'as': 14, 'for': 15, 'with': 16, 'movie': 17, 'but': 18, 'film': 19, "'s": 20, 'on': 21, 'you': 22, 'not': 23, 'are': 24, 'his': 25, 'he': 26, 'have': 27, 'be': 28, 'one': 29, 'all': 30, 'at': 31, 'by': 32, 'they': 33, 'an': 34, 'who': 35, 'so': 36, 'from': 37, 'like': 38, 'her': 39, "'t": 40, 'or': 41, 'just': 42, 'there': 43, 'about': 44, 'out': 45, "'": 46, 'has': 47, 'if': 48, 'some': 49, 'what': 50, 'good': 51, 'more': 52, 'very': 53, 'when': 54, 'she': 55, 'up': 56, 'can': 57, 'b': 58, 'time': 59, 'no': 60, 'even': 61, 'my': 62, 'would': 63, 'which': 64, 'story': 65, 'only': 66, 'really': 67, 'see': 68, 'their': 69, 'had': 70, 'were': 71, 'me': 72, 'well': 73, 'we': 74, 'than': 75, 'much': 76, 'been': 77, 'get': 78, 'bad': 79, 'will': 80, 'people': 81, 'do': 82, 'also': 83, 'other': 84, 'into': 85, 'because': 86, 'great': 87, 'first': 88, 'him': 89, 'how'

In [ ]:
from bs4 import BeautifulSoup
import string

stopwords = ['a', ... , 'yourselves']
table = str.maketrans('', '', string.punctuation)

imdb_sentences = []
train_data = tfds.as_numpy(tfds.load('imdb_reviews', split = 'train'))
for item in train_data:
  sentence = str(item['text'].decode('UTF-8')).lower())
  soup = BeautifulSoup(sentence)
  sentence = soup.get_text()
  words = sentence.split()
  for word in words:
    word = word.translate(table)
    if word not in stopwords:
      filtered_sentence = filtered_sentence + word + ' '
  imdb_sentences.append(filtered_sentence)

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 25000)
tokenizer.fit_on_texts(imdb_sentences)
sequences = tokenizer.texts_to_sequences(imdb_sentences)
print(tokenizer.word_index)

In [ ]:
#  him/her 이런 표현이 잘못 변환되는 것을 방지하기 위해 앞뒤로 공백 추가

sentence = sentence.replace(',', ' , ')
sentence = sentence.replace('-', ' - ')
sentence = sentence.replace('.', ' . ')
sentence = sentence.replace('/', ' / ')

In [ ]:
sentences = [
    'Today is a sunny day',
    'Today is a rainy day',
    'Is it sunny today?'
]
sequences = tokenizer.texts_to_sequences(sentences)
print(sentences)

In [ ]:
reverse_word_index = dict(
    [(value, key) for (key, value) in tokenizer.word_index.items()]
)

decoded_review = ' '.join([reverse_wod_index.get(i, '?') for i in sequences[0]])

print(decoded_review)

In [29]:
## 부분 단어 데이터셋 이용하기
# 말뭉치를 문자로 나누는 것(토큰 수는 적지만 의미가 부족)
# 말뭉치를 단어로 나누는 것(토큰 수는 많지만 의미가 풍부)
# 두 방법 사이의 절충안으로 부분 단어 사용

(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k',
    split = (tfds.Split.TRAIN, tfds.Split.TEST),
    as_supervised = True,
    with_info = True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteNTTKLN/imdb_reviews-train.tfrecord…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteNTTKLN/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteNTTKLN/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [30]:
encoder = info.features['text'].encoder
print('어휘 사전 크기: {}'.format(encoder.vocab_size))

어휘 사전 크기: 8185


In [31]:
print(encoder.subwords)

['the_', ', ', '. ', 'a_', 'and_', 'of_', 'to_', 's_', 'is_', 'br', 'in_', 'I_', 'that_', 'this_', 'it_', ' /><', ' />', 'was_', 'The_', 'as_', 't_', 'with_', 'for_', '.<', 'on_', 'but_', 'movie_', ' (', 'are_', 'his_', 'have_', 'film_', 'not_', 'ing_', 'be_', 'ed_', 'you_', ' "', 'it', 'd_', 'an_', 'he_', 'by_', 'at_', 'one_', 'who_', 'y_', 'from_', 'e_', 'or_', 'all_', 'like_', 'they_', '" ', 'so_', 'just_', 'has_', ') ', 'her_', 'about_', 'out_', 'This_', 'some_', 'ly_', 'movie', 'film', 'very_', 'more_', 'It_', 'would_', 'what_', 'when_', 'which_', 'good_', 'if_', 'up_', 'only_', 'even_', 'their_', 'had_', 'really_', 'my_', 'can_', 'no_', 'were_', 'see_', 'she_', '? ', 'than_', '! ', 'there_', 'get_', 'been_', 'into_', ' - ', 'will_', 'much_', 'story_', 'because_', 'ing', 'time_', 'n_', 'we_', 'ed', 'me_', ': ', 'most_', 'other_', 'don', 'do_', 'm_', 'es_', 'how_', 'also_', 'make_', 'its_', 'could_', 'first_', 'any_', "' ", 'people_', 'great_', 've_', 'ly', 'er_', 'made_', 'r_', 'B

In [32]:
sample_string = 'Today is a sunny day'

encoded_string = encoder.encode(sample_string)
print('인코딩된 문자열: {}'.format(encoded_string))

인코딩된 문자열: [6427, 4869, 9, 4, 2365, 1361, 606]


In [33]:
print(encoder.subwords[6426])

Tod


In [34]:
encoded_string = encoder.encode(sample_string)

original_string = encoder.decode(encoded_string)
test_string = encoder.decode([6427, 4869, 9, 4, 2365, 1361, 606])

In [ ]:
# csv 파일에서 텍스트 읽기

import csv

sentences = []
labels = []

with open('binary-emotion.csv', encoding = 'UTF-8') as csvfile:
  reader = csv.reader(csvfile, delimiter = ',')
  for row in reader:
    labels.append(int(row[0]))
    sentence = row[1].lower()
    sentence = sentence.replace(',', ' , ')
    sentence = sentence.replace('-', ' - ')
    sentence = sentence.replace('.', ' . ')
    sentence = sentence.replace('/', ' / ')
    soup = BeautifulSoup(sentence)
    sentence = soup.get_text()
    words = sentence.split()
    filtered_sentence = ""
    for word in words:
      word = word.translate(table)
      if word not in stopwords:
        filtered_sentence = filtered_sentence + word + ' '
    sentences.append(filtered_sentence)
  

In [ ]:
# 훈련 세트와 데이터 세트 만들기

training_size = 28000

training_sentences = sentences[0:training_size]
test_sentences = sentences[training_size:]
trainig_labels = labels[0:training_size]
test_labels = labels[training_size:]

In [ ]:
vocab_size = 20000
max_length = 10
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)

training_padded = pad_sequences(training_sequences, maxlen = max_length,
                                padding = padding_type, truncating = trunc_type)


In [ ]:
print(training_sequences[0])
print(training_padded[0])

In [ ]:
print(tokenizer.word_index)

In [ ]:
# JSON 파일에서 텍스트 읽기
# 캐글의 news headlines dataset for sarcasm Detection

import json
with open('sarcasm.json', 'r') as f:
  datastore = json.load(f)
  for item in datastore:
    sentence = item['headline'].lower()
    label = item['is_sarcastic']
    link = item['article_link']

In [ ]:
# csv 파일에서 텍스트 읽기

with open('sarcasm.json', 'r') as f:
  datastore = json.load(f)

sentences = []
labels = []
urls = []

for item in datastore:
    sentence = item['headline'].lower()
    sentence = sentence.replace(',', ' , ')
    sentence = sentence.replace('-', ' - ')
    sentence = sentence.replace('.', ' . ')
    sentence = sentence.replace('/', ' / ')
    soup = BeautifulSoup(sentence)
    sentence = soup.get_text()
    words = sentence.split()
    filtered_sentence = ""
    for word in words:
      word = word.translate(table)
      if word not in stopwords:
        filtered_sentence = filtered_sentence + word + ' '
    sentences.append(filtered_sentence)
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])

In [ ]:
training_size = 23000

training_sentences = sentences[0:training_size]
test_sentences = sentences[training_size:]
trainig_labels = labels[0:training_size]
test_labels = labels[training_size:]

In [ ]:
vocab_size = 20000
max_length = 10
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)

training_padded = pad_sequences(training_sequences, maxlen = max_length,
                                padding = padding_type, truncating = trunc_type)
print(word_index)